In [1]:
import sys
print(sys.executable)

/Users/jasmine.frantz/.local/pipx/venvs/notebook/bin/python


In [3]:
!pip install torch transformers

In [4]:
import sys
print(f"Python path: {sys.executable}")
!{sys.executable} -m pip list | grep torch
!{sys.executable} -m pip list | grep transformers


Python path: /Users/jasmine.frantz/.local/pipx/venvs/notebook/bin/python


In [11]:
import sys
!{sys.executable} -m pip install torch transformers


  Using cached torch-2.6.0-cp313-none-macosx_11_0_arm64.whl.metadata (28 kB)
  Using cached transformers-4.49.0-py3-none-any.whl.metadata (44 kB)
  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached networkx-3.4.2-py3-none-any.whl.metadata (6.3 kB)
  Using cached fsspec-2025.3.0-py3-none-any.whl.metadata (11 kB)
  Using cached sympy-1.13.1-py3-none-any.whl.metadata (12 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
  Using cached huggingface_hub-0.29.3-py3-none-any.whl.metadata (13 kB)
  Using cached numpy-2.2.4-cp313-cp313-macosx_14_0_arm64.whl.metadata (62 kB)
  Using cached regex-2024.11.6-cp313-cp313-macosx_11_0_arm64.whl.metadata (40 kB)
  Using cached tokenizers-0.21.1-cp39-abi3-macosx_11_0_arm64.whl.metadata (6.8 kB)
  Using cached safetensors-0.5.3-cp38-abi3-macosx_11_0_arm64.whl.metadata (3.8 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
Using cached torch-2.6.0-cp313-none-macosx_11_0_arm64.whl (66.5 MB)
Using 

In [12]:
# beforehand, install torch transformers pip install torch transformers
import os
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Disable tokenizers parallelism warning
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# Automatically detect the correct device (MPS for Apple Silicon)
device = "mps" if torch.backends.mps.is_available() else "cpu"

# Load model and tokenizer
model_name = "facebook/opt-350m"  # Replace with your model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)  # Move model to MPS

# Prepare input text and move input_ids to the same device
input_text = "The meaning of life is"
inputs = tokenizer(input_text, return_tensors="pt")  # Convert input text to tensor
# inputs is not a tensor but a BatchEncoding object.
# To extract the actual tensor, you need to do:
input_ids = inputs["input_ids"].to(device)  # Extract `input_ids` and move to MPS

# Generate output
with torch.no_grad():
    output = model.generate(input_ids)

# Decode and print output
print(tokenizer.decode(output[0], skip_special_tokens=True))


/Users/jasmine.frantz/.local/pipx/venvs/notebook/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The meaning of life is to be happy.
I'm not sure if you're being sarcastic or not.
I'm


In [13]:
input_ids

tensor([[   2,  133, 3099,    9,  301,   16]], device='mps:0')

In [14]:
inputs = tokenizer("The meaning of life is", return_tensors="pt")
print(inputs)


{'input_ids': tensor([[   2,  133, 3099,    9,  301,   16]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1]])}


In [15]:
del model


In [16]:
!pip install modal

In [18]:
import sys
!{sys.executable} -m pip install modal


  Using cached modal-0.73.116-py3-none-any.whl.metadata (2.5 kB)
  Using cached aiohttp-3.11.14-cp313-cp313-macosx_11_0_arm64.whl.metadata (7.7 kB)
  Using cached click-8.1.8-py3-none-any.whl.metadata (2.3 kB)
  Using cached fastapi-0.115.11-py3-none-any.whl.metadata (27 kB)
  Using cached grpclib-0.4.7-py3-none-any.whl
  Using cached protobuf-5.29.4-cp38-abi3-macosx_10_9_universal2.whl.metadata (592 bytes)
  Using cached rich-13.9.4-py3-none-any.whl.metadata (18 kB)
  Using cached synchronicity-0.9.11-py3-none-any.whl.metadata (13 kB)
  Using cached toml-0.10.2-py2.py3-none-any.whl.metadata (7.1 kB)
  Using cached typer-0.15.2-py3-none-any.whl.metadata (15 kB)
  Using cached types_certifi-2021.10.8.3-py3-none-any.whl.metadata (1.4 kB)
  Using cached types_toml-0.10.8.20240310-py3-none-any.whl.metadata (1.5 kB)
  Using cached watchfiles-1.0.4-cp313-cp313-macosx_11_0_arm64.whl.metadata (4.9 kB)
  Using cached h2-4.2.0-py3-none-any.whl.metadata (5.1 kB)
  Using cached multidict-6.2.0-cp3

In [20]:
import modal
from transformers import AutoTokenizer, AutoModelForCausalLM

# Create a Modal image using Debian slim and install required dependencies
# Original:
# image = modal.Image.debian_slim().pip_install("fastapi[standard]", "transformers")

# Updated:
image = modal.Image.debian_slim().pip_install(
    "fastapi[standard]",
    "transformers",
    "torch>=2.0.0",
    "accelerate",
    "safetensors"  # Optional but recommended for faster model loading
)

# Add local Python modules explicitly to avoid automounting warning
image_with_source = image.add_local_python_source("_remote_module_non_scriptable")

# Initialize a Modal App with the custom image
# Original:
# app = modal.App(name="example-lifecycle-web", image=image)

# Updated:
app = modal.App(name="example-lifecycle-web", image=image_with_source)

# Define a stub class for your model
class MyModel:
    def __init__(self):
        # Load the tokenizer and model once during initialization
        self.tokenizer = AutoTokenizer.from_pretrained("facebook/opt-125m")
        self.model = AutoModelForCausalLM.from_pretrained("facebook/opt-125m", device_map="auto")
    
    def run_inference(self, input_text: str) -> str:
        # Perform inference and return the result
        input_ids = self.tokenizer(input_text, return_tensors="pt")
        outputs = self.model.generate(**input_ids)
        return self.tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

# Define the web endpoint to accept requests and run inference
@app.function()
# Original:
# @modal.web_endpoint()  # Expose this function via an HTTP endpoint

# Updated:
@modal.fastapi_endpoint()  # Updated from web_endpoint to fastapi_endpoint
def hello(input_text: str) -> str:
    # Initialize the model here, it will persist between calls
    if not hasattr(hello, "model_instance"): # Only initialize on first call
        hello.model_instance = MyModel()
    result = hello.model_instance.run_inference(input_text)  # Run inference
    return result

# To run the web app
if __name__ == "__main__":
    app.run()